# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 10, 27, 17, 15

while True:
    dt = datetime.now()
    if dt.month >= month:
        if dt.day > day:
            break
        if dt.day == day and (dt.hour > hour or (dt.hour == hour and dt.minute > minute)):
            break
    sleep(60)

# Parameter optimization

In [2]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 300
load = False

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'RSI': {'timeperiod': [12, 14, 16], 'low_bound': [20, 25, 30, 35]},
              'STOCH': {'fastk_period': [3, 5, 7, 9, 11], 'slowk_period': [2, 3, 4, 5],
                        'slowd_period': [2, 3, 5, 7, 9], 'low_bound': [10, 15, 20, 25]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Number of combinations is 4800


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4800/4800 [3:45:26<00:00,  2.82s/it]


# Check global statistics

In [3]:
from glob import glob

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '4h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
stat.groupby(['RSI_timeperiod', 
              'RSI_low_bound', 
              'STOCH_fastk_period', 
              'STOCH_slowk_period', 
              'STOCH_slowd_period', 
              'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                       'pct_price_diff_avg': 'mean',
                                       'forecast_rank': 'mean', 
                                       'price_rank': 'mean', 
                                       'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                            ascending=False).head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
14             35            3                  3                  9                  25                            80.253750   
12             35            3                  4                  9                  25                            81.250000   
14             35            3                  4                  9                  25                            81.250000   
12             35            3                  3                  9                  25                            79.167083   
16             35            3                  3                  9                  25                            79.355417   
                                                4                  9                  25                            80.483750   
12             30            3                  4                  9                  25                            79.398333   
16             15            5                  4                  3                  25                            93.229167   
                             7                  4                  5                  25                            95.025694   
12             30            3                  3                  9                  25                            77.291667   
16             20            11                 4                  9                  25                            75.911146   
               15            7                  4                  3                  25                            92.319861   
                                                                                      20                            92.592639   
12             35            11                 5                  2                  10                            71.017083   
16             20            11                 4                  9                  20                            68.006250   
               15            5                  4                  3                  20                            92.778194   
                             7                  3                  5                  25                            92.466806   
14             30            3                  4                  9                  25                            79.807500   
16             15            7                  5                  3                  25                            92.336528   
                                                                                      20                            92.796806   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
14             35            3                  3                  9                  25                        -1.446667   
12             35            3                  4                  9                  25                        -1.247917   
14             35            3                  4                  9                  25                        -1.247917   
12             35            3                  3                  9                  25                        -1.395833   
16             35            3                  3                  9                  25                        -1.400417   
                                                4                  9                  25                        -1.273750   
12             30            3                  4                  9                  25                        -1.224167   
16             15            5                  4                  3                  25                        -1.893472   
              

# Save new config data to config file

In [19]:
from config_updater import ConfigUpdater

timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'RSI': {'timeperiod': [14], 'low_bound': [35]},
                  'STOCH': {'fastk_period': [3], 'slowk_period': [3],
                            'slowd_period': [2], 'low_bound': [20]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)